# An Example of Inference Accuracy Check

This tutorial will explain how to compare the inference accuracies between furiosa-sdk using NPU and other runtimes using CPU or GPU. In this example, we are going to use Onnx runtime as a counterpart runtime.

## Prerequisites
To follow this tutorial, please install the followings first.

To run this examples, you must install the required packages and setup Python environment by following the guides:
* [FuriosaAI Driver, Firmware, Runtime Installation Guide](https://furiosa-ai.github.io/docs/latest/ko/software/installation.html)
* [Setting up a Python Environment](https://furiosa-ai.github.io/docs/latest/ko/software/python-sdk.html#python)

Then, please install the following python packages:
```sh
pip install furiosa-sdk matplotlib mnist onnxruntime
```

Or, you can run the following command to install all dependent packages for all notebook examples at once:
```sh
pip install -r examples/notebooks/requirements.txt
```

And then, let's check if your NPU device is ready as following:

In [ ]:
!furiosactl info

Then, let's make sure that your SDK is ready to run.

In [ ]:
!python -c "from furiosa import runtime;print(runtime.__full_version__)"

## Preparing the dataset and model

In [4]:
# download mnist dataset
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2024-11-05 16:49:43--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2024-11-05 16:49:44--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.2’

MNIST.tar.gz.2          [              <=>   ]  33.20M  6.60MB/s    in 5.3s    

2024-11-05 16:49:50 (6.27 MB/s) - ‘MNIST.tar.gz.2’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

In [5]:
# Import MNIST dataset package
import numpy as np
from mnist import MNIST

mndata = MNIST('./MNIST/raw/')
train_images, train_labels = mndata.load_training()

mnist_images = np.array(train_images).reshape((60000, 1, 28, 28)).astype(np.float32)
mnist_images.shape

(60000, 1, 28, 28)

In [10]:
from pathlib import Path

model_path = 'models/mnist-8.onnx'

In [11]:
import onnxruntime

onnxrt = onnxruntime.InferenceSession(model_path)

In [12]:
from furiosa.runtime.sync import create_runner

runner = create_runner(model_path)
runner.model.print_summary()

2024-11-05T07:50:57.487826Z  INFO furiosa_rt_core::driver::event_driven::coord: FuriosaRT (v0.10.3, rev: 394c19392, built at: 2023-11-22T08:53:04Z) bootstrapping ...
2024-11-05T07:50:57.490217Z  INFO furiosa_rt_core::driver::event_driven::coord: Found furiosa-compiler (v0.10.1, rev: 8b00177, built at: 2024-05-28T06:18:01Z)
2024-11-05T07:50:57.490222Z  INFO furiosa_rt_core::driver::event_driven::coord: Found libhal (type: warboy, v0.12.0, rev: 56530c0 built at: 2023-11-16T12:34:03Z)
2024-11-05T07:50:57.490224Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-1] detected 1 NPU device(s):
2024-11-05T07:50:57.494907Z  INFO furiosa_rt_core::driver::event_driven::coord: - [0] npu:8:0-1 (warboy-b0-2pe, 128dpes, firmware: 1.7.8, e9f371e)
2024-11-05T07:50:57.495084Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-1] started
2024-11-05T07:50:57.496167Z  INFO furiosa::runtime: Saving the compilation log into /home/changmin/.local/state/furiosa/logs/compiler-20241105165057

In [13]:
# to compare floating point numbers with tolerance
def numpy_equals(expected, result, atol=0.1):
    return np.allclose(expected, result, atol=atol), "{} was expected, but the result was {}".format(
        expected, result
    )

The following function runs inferences on both CPU and NPU and compree the results. It will take some time depending on `total_run`.

In [15]:
%%time

import random
total_run = 20 # How many inferences are compared
matched = 0

for _ in range(0, total_run):
    # randomly picks the item
    idx = random.randrange(0, 60000, 1)
    ndarray_value = mnist_images[idx : idx + 1]
    
    result1 = runner.run(ndarray_value)
    result2 = onnxrt.run(["Plus214_Output_0"], {"Input3": ndarray_value})    
    
    if numpy_equals(result1[0], result2, 0.04):
        matched += 1
        
print(f"Accuracy: {matched / total_run * 100}%")

Accuracy: 100.0%
CPU times: user 31 s, sys: 198 ms, total: 31.2 s
Wall time: 490 ms


In [16]:
# Close the session after you use.
runner.close()

2024-11-05T07:52:03.778702Z  INFO furiosa_rt_core::driver::event_driven::coord: [Sess-132e0b17] terminated
2024-11-05T07:52:03.786938Z  INFO furiosa_rt_core::npu::raw: NPU (npu:8:0-1) has been closed
2024-11-05T07:52:03.788231Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-1] stopped


True